In [11]:
import tensorflow as tf
tf.__version__

'2.8.0'

In [1]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt
import imgaug

In [2]:
# Root directory of the project
ROOT_DIR = "/home/brin/Rahma/SpeedBumpDetection"

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

In [3]:
#from tensorflow.keras.mixed_precision import experimental as mixed_precision

#policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_policy(policy)

In [4]:
# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")



class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"


    # NUMBER OF GPUs to use. When using only a CPU, this needs to be set to 1.
    GPU_COUNT = 1
    
    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2
    
    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + Speed Bump

    # Number of training steps per epoch
    # Jika dataset sedikit pake nilai yang kecil. Jika dataset banyak pake nilai yang lebih besar misal 10-20
    STEPS_PER_EPOCH = 5

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
    
    LEARNING_RATE = 0.001

In [5]:
############################################################
#  Dataset
############################################################

class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        """Load a subset of the Dog-Cat dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
      
        # Add classes. We have only one class to add.
        self.add_class("object", 1, "speedbump")


     
        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Load annotations
        # VGG Image Annotator saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        annotations1 = json.load(open('/home/brin/Rahma/SpeedBumpDetection/data/train/train.json'))
        # print(annotations1)
        annotations = list(annotations1.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:
            # print(a)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['names'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"speedbump": 1}

            # key = tuple(name_dict)
            num_ids = [name_dict[a] for a in objects]
     
            # num_ids = [int(n['Event']) for n in objects]
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a Dog-Cat dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [6]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom("/home/brin/Rahma/SpeedBumpDetection/data", "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom("/home/brin/Rahma/SpeedBumpDetection/data", "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=100,
                layers='heads')
                
#    model.train(dataset_train, dataset_val,
#                learning_rate=config.LEARNING_RATE,
#                epochs=20,
#                layers='heads', #layers='all', 
#                augmentation = imgaug.augmenters.Sequential([ 
#                imgaug.augmenters.Fliplr(1), 
#                imgaug.augmenters.Flipud(1), 
#                imgaug.augmenters.Affine(rotate=(-45, 45)), 
#                imgaug.augmenters.Affine(rotate=(-90, 90)), 
#                imgaug.augmenters.Affine(scale=(0.5, 1.5)),
#                imgaug.augmenters.Crop(px=(0, 10)),
#                imgaug.augmenters.Grayscale(alpha=(0.0, 1.0)),
#                imgaug.augmenters.AddToHueAndSaturation((-20, 20)), # change hue and saturation
#                imgaug.augmenters.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
#                imgaug.augmenters.Invert(0.05, per_channel=True), # invert color channels
#                imgaug.augmenters.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images               
#                ]
#                
#                ))
				

'''
 this augmentation is applied consecutively to each image. In other words, for each image, the augmentation apply flip LR,
 and then followed by flip UD, then followed by rotation of -45 and 45, then followed by another rotation of -90 and 90,
 and lastly followed by scaling with factor 0.5 and 1.5. '''
	
    
# Another way of using imgaug    
# augmentation = imgaug.Sometimes(5/6,aug.OneOf(
                                            # [
                                            # imgaug.augmenters.Fliplr(1), 
                                            # imgaug.augmenters.Flipud(1), 
                                            # imgaug.augmenters.Affine(rotate=(-45, 45)), 
                                            # imgaug.augmenters.Affine(rotate=(-90, 90)), 
                                            # imgaug.augmenters.Affine(scale=(0.5, 1.5))
                                             # ]
                                        # ) 
                                   # )
                                    

    
				
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

weights_path = COCO_WEIGHTS_PATH
        # Download weights file
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

In [7]:
train(model)

objects: ['speedbump']
numids [1]
objects: ['speedbump']
numids [1]
objects: ['speedbump']
numids [1]
objects: ['speedbump']
numids [1]
objects: ['speedbump', 'speedbump']
numids [1, 1]
objects: ['speedbump']
numids [1]
objects: ['speedbump']
numids [1]
objects: ['speedbump']
numids [1]
objects: ['speedbump']
numids [1]
objects: ['speedbump']
numids [1]
objects: ['speedbump']
numids [1]
objects: ['speedbump']
numids [1]
objects: ['speedbump', 'speedbump']
numids [1, 1]
objects: ['speedbump']
numids [1]
objects: ['speedbump']
numids [1]
objects: ['speedbump']
numids [1]
Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: /home/brin/Rahma/SpeedBumpDetection/logs/object20240206T1432/mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4   

/home/brin/anaconda3/envs/rahma/lib/python3.9/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/100


/home/brin/anaconda3/envs/rahma/lib/python3.9/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/home/brin/anaconda3/envs/rahma/lib/python3.9/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/home/brin/anaconda3/envs/rahma/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD

1/5 [=====>........................] - ETA: 33s - batch: 0.0000e+00 - size: 2.0000 - loss: 11.9227 - rpn_class_loss: 8.5605 - rpn_bbox_loss: 1.3608 - mrcnn_class_loss: 0.0643 - mrcnn_bbox_loss: 1.2334 - mrcnn_mask_loss: 0.7038

...

4/5 [=======================>......] - ETA: 0s - batch: 1.5000 - size: 2.0000 - loss: 0.3892 - rpn_class_loss: 0.0042 - rpn_bbox_loss: 0.1043 - mrcnn_class_loss: 0.0072 - mrcnn_bbox_loss: 0.1263 - mrcnn_mask_loss: 0.1472

/home/brin/anaconda3/envs/rahma/lib/python3.9/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/home/brin/anaconda3/envs/rahma/lib/python3.9/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)


5/5 [==============================] - ETA: 0s - batch: 2.0000 - size: 2.0000 - loss: 0.3628 - rpn_class_loss: 0.0042 - rpn_bbox_loss: 0.0897 - mrcnn_class_loss: 0.0082 - mrcnn_bbox_loss: 0.1108 - mrcnn_mask_loss: 0.1499

/home/brin/anaconda3/envs/rahma/lib/python3.9/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/home/brin/anaconda3/envs/rahma/lib/python3.9/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/home/brin/anaconda3/envs/rahma/lib/python3.9/site-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set or

5/5 [==============================] - 6s 1s/step - batch: 2.0000 - size: 2.0000 - loss: 0.3628 - rpn_class_loss: 0.0042 - rpn_bbox_loss: 0.0897 - mrcnn_class_loss: 0.0082 - mrcnn_bbox_loss: 0.1108 - mrcnn_mask_loss: 0.1499 - val_loss: 0.3442 - val_rpn_class_loss: 0.0078 - val_rpn_bbox_loss: 0.0935 - val_mrcnn_class_loss: 0.0076 - val_mrcnn_bbox_loss: 0.0781 - val_mrcnn_mask_loss: 0.1572
